In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

ModuleNotFoundError: No module named 'tensorflow'

In [3]:
# Load your stock price data (replace this with your data source)
# For this example, let's assume you have a CSV file with columns: Date, Close_Price
data = pd.read_csv('stock_data.csv')
close_prices = data['Close_Price'].values.reshape(-1, 1)


FileNotFoundError: [Errno 2] No such file or directory: 'stock_data.csv'

In [4]:
# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
normalized_data = scaler.fit_transform(close_prices)

# Split the data into training and testing sets
train_size = int(len(normalized_data) * 0.8)
train_data, test_data = normalized_data[:train_size], normalized_data[train_size:]


NameError: name 'close_prices' is not defined

In [5]:
# Create sequences and labels for training
def create_sequences(data, seq_length):
    sequences = []
    labels = []
    for i in range(len(data) - seq_length):
        seq = data[i:i + seq_length]
        label = data[i + seq_length]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

seq_length = 10  # Length of the input sequences
train_sequences, train_labels = create_sequences(train_data, seq_length)
test_sequences, test_labels = create_sequences(test_data, seq_length)

# Build the LSTM model
model = Sequential()
model.add(LSTM(50, input_shape=(seq_length, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(train_sequences, train_labels, epochs=50, batch_size=32)


NameError: name 'train_data' is not defined

In [ ]:
# Make predictions
train_predictions = model.predict(train_sequences)
test_predictions = model.predict(test_sequences)

# Inverse transform to get the actual prices
train_predictions = scaler.inverse_transform(train_predictions)
test_predictions = scaler.inverse_transform(test_predictions)
train_labels = scaler.inverse_transform(train_labels)
test_labels = scaler.inverse_transform(test_labels)

# Calculate Mean Squared Error
train_rmse = np.sqrt(mean_squared_error(train_labels, train_predictions))
test_rmse = np.sqrt(mean_squared_error(test_labels, test_predictions))

print(f"Train RMSE: {train_rmse}")
print(f"Test RMSE: {test_rmse}")

# Plot the results
plt.figure(figsize=(12, 6))
plt.plot(data['Date'][:len(train_data)], train_labels, label='Actual Train Prices')
plt.plot(data['Date'][len(train_data) + seq_length:], test_labels, label='Actual Test Prices')
plt.plot(data['Date'][:len(train_data) + seq_length], np.concatenate((train_labels[-seq_length:], test_predictions)), label='Predicted Prices')
plt.legend()
plt.show()